In [1]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F

Using backend: pytorch


In [2]:
import dgl.data

dataset = dgl.data.CoraGraphDataset()
print('Number of categories:', dataset.num_classes)

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
Number of categories: 7


In [3]:
g = dataset[0]

In [4]:
print('Node features')
print(g.ndata)
print('Edge features')
print(g.edata)

Node features
{'feat': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]), 'label': tensor([3, 4, 4,  ..., 3, 3, 3]), 'test_mask': tensor([False, False, False,  ...,  True,  True,  True]), 'train_mask': tensor([ True,  True,  True,  ..., False, False, False]), 'val_mask': tensor([False, False, False,  ..., False, False, False])}
Edge features
{}


In [5]:
from dgl.nn import GraphConv

class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes, num_layers=2):
        super(GCN, self).__init__()
        self.convs = nn.ModuleList()
        self.num_layers = num_layers
        
        self.convs.append(GraphConv(in_feats, h_feats))
        for _ in range(num_layers-2):
            self.convs.append(GraphConv(h_feats, h_feats))
        self.convs.append(GraphConv(h_feats, num_classes))
    
    def forward(self, g, h):
        for ell in range(self.num_layers-1):
            h = self.convs[ell](g, h)
            h = F.relu(h)            
        h = self.convs[-1](g, h)
        return h
    

DGL provides implementation of many popular neighbor aggregation
modules. You can easily invoke them with one line of code.




Training the GCN
----------------

Training this GCN is similar to training other PyTorch neural networks.




In [6]:
def train(g, model, iters):
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    best_val_acc = 0
    best_test_acc = 0

    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    val_mask = g.ndata['val_mask']
    test_mask = g.ndata['test_mask']
    for e in range(iters):
        # Forward
        logits = model(g, features)

        # Compute prediction
        pred = logits.argmax(1)

        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % 5 == 0:
            print('In epoch {}, loss: {:.3f}, train acc: {:.3f}, val acc: {:.3f} (best {:.3f}), test acc: {:.3f} (best {:.3f})'.format(
                e, loss, train_acc, val_acc, best_val_acc, test_acc, best_test_acc))
            
# Create the model with given dimensions
model = GCN(in_feats=g.ndata['feat'].shape[1], 
            h_feats=64, 
            num_classes=dataset.num_classes, 
            num_layers=8)

print(model)

train(g, model, iters=200)

GCN(
  (convs): ModuleList(
    (0): GraphConv(in=1433, out=64, normalization=both, activation=None)
    (1): GraphConv(in=64, out=64, normalization=both, activation=None)
    (2): GraphConv(in=64, out=64, normalization=both, activation=None)
    (3): GraphConv(in=64, out=64, normalization=both, activation=None)
    (4): GraphConv(in=64, out=64, normalization=both, activation=None)
    (5): GraphConv(in=64, out=64, normalization=both, activation=None)
    (6): GraphConv(in=64, out=64, normalization=both, activation=None)
    (7): GraphConv(in=64, out=7, normalization=both, activation=None)
  )
)
In epoch 0, loss: 1.946, train acc: 0.136, val acc: 0.280 (best 0.280), test acc: 0.298 (best 0.298)
In epoch 5, loss: 1.944, train acc: 0.257, val acc: 0.246 (best 0.280), test acc: 0.242 (best 0.298)
In epoch 10, loss: 1.938, train acc: 0.343, val acc: 0.350 (best 0.350), test acc: 0.366 (best 0.366)
In epoch 15, loss: 1.924, train acc: 0.336, val acc: 0.372 (best 0.372), test acc: 0.382 (bes

In [7]:
# Thumbnail credits: Stanford CS224W Notes
# sphinx_gallery_thumbnail_path = '_static/blitz_1_introduction.png'